# Adir1 cloud data access

This notebook provides information about how to download data from the [MalariaGEN Vector Observatory Asian Vector Genomic Surveillance Project](https://www.malariagen.net/project/vector-observatory-asia/), for *Anopheles dirus* via Google Cloud. This includes sample metadata, raw sequence reads, sequence read alignments, and single nucleotide polymorphism (SNP) calls. 

This notebook illustrates how to read data directly from the cloud, without having to first download any data locally. This notebook can be run from any computer, but will work best when run from a compute node within Google Cloud, because it will be physically closer to the data and so data transfer is faster. For example, this notebook can be run via [Google Colab](https://colab.research.google.com/) which are free interactive computing service running in the cloud.

To launch this notebook in the cloud and run it for yourself, click the launch icon (<i class="fas fa-rocket"></i>) at the top of the page and select one of the cloud computing services available.

## Data hosting

All data required for this notebook is hosted on Google Cloud Storage (GCS). Data are hosted in the `vo_adir_production_us_central1` bucket, which is a single-region bucket located in the United States. All data hosted in GCS are publicly accessible and do not require any authentication to access. 

## Setup

Running this notebook requires some Python packages to be installed:

In [1]:
%pip install -q malariagen_data

Note: you may need to restart the kernel to use updated packages.


To make accessing these data more convenient, we've created the [malariagen_data](https://github.com/malariagen/malariagen-data-python) Python package. This is experimental so please let us know if you find any bugs or have any suggestions. See the [Af1 API docs](https://malariagen.github.io/malariagen-data-python/latest/Af1.html) for documentation of all functions available from this package. 

Import other packages we'll need to use here.

In [2]:
import numpy as np
import dask
import dask.array as da
from dask.diagnostics.progress import ProgressBar
# silence some warnings
dask.config.set(**{'array.slicing.split_large_chunks': False})
import allel
import malariagen_data

`Adir1` data access from Google Cloud is set up with the following code:

In [ ]:
adir1 = malariagen_data.Adir1()
adir1

<MalariaGEN Af1 API client>
Storage URL             : gs://vo_afun_release_master_us_central1
Data releases available : 1.0
Results cache           : None
Cohorts analysis        : 20231215
Site filters analysis   : dt_20200416
Software version        : malariagen_data 10.0.0
Client location         : Iowa, United States (Google Cloud us-central1)
---
Please note that data are subject to terms of use,
for more information see https://www.malariagen.net/data
or contact support@malariagen.net. For API documentation see 
https://malariagen.github.io/malariagen-data-python/v10.0.0/Af1.html

## Sample sets

Data are organised into different releases. As an example, data in Af1.0 are organised into 8 sample sets. Each of these sample sets corresponds to a set of mosquito specimens contributed by a collaborating study. Depending on your objectives, you may want to access data from only specific sample sets, or all sample sets.

To see which sample sets are available, load the sample set manifest into a pandas dataframe:

In [ ]:
df_sample_sets = adir1.sample_sets(release="1.0")
df_sample_sets

,sample_set,sample_count,study_id,study_url,release
0,1229-VO-GH-DADZIE-VMF00095,36,1229-VO-GH-DADZIE,https://www.malariagen.net/network/where-we-wo...,1.0
1,1230-VO-GA-CF-AYALA-VMF00045,50,1230-VO-MULTI-AYALA,https://www.malariagen.net/network/where-we-wo...,1.0
2,1231-VO-MULTI-WONDJI-VMF00043,320,1231-VO-MULTI-WONDJI,https://www.malariagen.net/network/where-we-wo...,1.0
3,1232-VO-KE-OCHOMO-VMF00044,81,1232-VO-KE-OCHOMO,https://www.malariagen.net/network/where-we-wo...,1.0
4,1235-VO-MZ-PAAIJMANS-VMF00094,76,1235-VO-MZ-PAAIJMANS,https://www.malariagen.net/network/where-we-wo...,1.0
5,1236-VO-TZ-OKUMU-VMF00090,10,1236-VO-TZ-OKUMU,https://www.malariagen.net/network/where-we-wo...,1.0
6,1240-VO-CD-KOEKEMOER-VMF00099,43,1240-VO-MULTI-KOEKEMOER,https://www.malariagen.net/network/where-we-wo...,1.0
7,1240-VO-MZ-KOEKEMOER-VMF00101,40,1240-VO-MULTI-KOEKEMOER,https://www.malariagen.net/network/where-we-wo...,1.0


For more information about these sample sets, you can read about each sample set from the URLs under the field `study_url`.

## Sample metadata

Data about the samples that were sequenced to generate this data resource are available, including the time and place of collection, the gender of the specimen, and our call regarding the species of the specimen. These are organised by sample set.

E.g., load sample metadata for all samples in the Af1.0 release into a [pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/user_guide/dsintro.html#dataframe):

In [ ]:
df_samples = adir1.sample_metadata(sample_sets="1.0")
df_samples

,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call,...,admin1_name,admin1_iso,admin2_name,taxon,cohort_admin1_year,cohort_admin1_month,cohort_admin1_quarter,cohort_admin2_year,cohort_admin2_month,cohort_admin2_quarter
0,VBS24195,1229-GH-A-GH01,Samuel Dadzie,Ghana,Dimabi,2017,8,9.420,-1.083,F,...,Northern Region,GH-NP,Tolon,funestus,GH-NP_fune_2017,GH-NP_fune_2017_08,GH-NP_fune_2017_Q3,GH-NP_Tolon_fune_2017,GH-NP_Tolon_fune_2017_08,GH-NP_Tolon_fune_2017_Q3
1,VBS24196,1229-GH-A-GH02,Samuel Dadzie,Ghana,Gbullung,2017,7,9.488,-1.009,F,...,Northern Region,GH-NP,Kumbungu,funestus,GH-NP_fune_2017,GH-NP_fune_2017_07,GH-NP_fune_2017_Q3,GH-NP_Kumbungu_fune_2017,GH-NP_Kumbungu_fune_2017_07,GH-NP_Kumbungu_fune_2017_Q3
2,VBS24197,1229-GH-A-GH03,Samuel Dadzie,Ghana,Dimabi,2017,7,9.420,-1.083,F,...,Northern Region,GH-NP,Tolon,funestus,GH-NP_fune_2017,GH-NP_fune_2017_07,GH-NP_fune_2017_Q3,GH-NP_Tolon_fune_2017,GH-NP_Tolon_fune_2017_07,GH-NP_Tolon_fune_2017_Q3
3,VBS24198,1229-GH-A-GH04,Samuel Dadzie,Ghana,Dimabi,2017,8,9.420,-1.083,F,...,Northern Region,GH-NP,Tolon,funestus,GH-NP_fune_2017,GH-NP_fune_2017_08,GH-NP_fune_2017_Q3,GH-NP_Tolon_fune_2017,GH-NP_Tolon_fune_2017_08,GH-NP_Tolon_fune_2017_Q3
4,VBS24199,1229-GH-A-GH05,Samuel Dadzie,Ghana,Gupanarigu,2017,8,9.497,-0.952,F,...,Northern Region,GH-NP,Kumbungu,funestus,GH-NP_fune_2017,GH-NP_fune_2017_08,GH-NP_fune_2017_Q3,GH-NP_Kumbungu_fune_2017,GH-NP_Kumbungu_fune_2017_08,GH-NP_Kumbungu_fune_2017_Q3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,VBS24534,1240-MZ-A-MozF_1314,Lizette Koekemoer,Mozambique,Motinho,2015,8,-10.851,40.594,F,...,Cabo Delgado,MZ-P,Palma,funestus,MZ-P_fune_2015,MZ-P_fune_2015_08,MZ-P_fune_2015_Q3,MZ-P_Palma_fune_2015,MZ-P_Palma_fune_2015_08,MZ-P_Palma_fune_2015_Q3
652,VBS24535,1240-MZ-A-MozF_1315,Lizette Koekemoer,Mozambique,Motinho,2015,8,-10.851,40.594,F,...,Cabo Delgado,MZ-P,Palma,funestus,MZ-P_fune_2015,MZ-P_fune_2015_08,MZ-P_fune_2015_Q3,MZ-P_Palma_fune_2015,MZ-P_Palma_fune_2015_08,MZ-P_Palma_fune_2015_Q3
653,VBS24536,1240-MZ-A-MozF_1317,Lizette Koekemoer,Mozambique,Motinho,2015,8,-10.851,40.594,F,...,Cabo Delgado,MZ-P,Palma,funestus,MZ-P_fune_2015,MZ-P_fune_2015_08,MZ-P_fune_2015_Q3,MZ-P_Palma_fune_2015,MZ-P_Palma_fune_2015_08,MZ-P_Palma_fune_2015_Q3
654,VBS24537,1240-MZ-A-MozF_1319,Lizette Koekemoer,Mozambique,Motinho,2015,8,-10.851,40.594,F,...,Cabo Delgado,MZ-P,Palma,funestus,MZ-P_fune_2015,MZ-P_fune_2015_08,MZ-P_fune_2015_Q3,MZ-P_Palma_fune_2015,MZ-P_Palma_fune_2015_08,MZ-P_Palma_fune_2015_Q3


The `sample_id` column gives the sample identifier used throughout all Af1 analyses.

The `country`, `location`, `latitude` and `longitude` columns give the location where the specimen was collected.

The `year` and `month` columns give the approximate date when the specimen was collected.

The `sex_call` column gives the gender as determined from the sequence data.

[Pandas](https://pandas.pydata.org/) can be used to explore and query the sample metadata in various ways. E.g., here is a summary of the numbers of samples by species:

In [5]:
df_samples.groupby("taxon").size()

taxon
funestus    656
dtype: int64

## SNP calls

Data on SNP calls, including the SNP positions, alleles, site filters, and genotypes, can be accessed as an [xarray Dataset](http://xarray.pydata.org/en/stable/user-guide/data-structures.html#dataset).

E.g., access SNP calls for chromosome 2RL for all samples in `Af1.0`.

In [ ]:
ds_snps = adir1.snp_calls(region="2RL", sample_sets="1.0")
ds_snps

<xarray.Dataset> Size: 1TB
Dimensions:                       (variants: 102882611, alleles: 4,
                                   samples: 656, ploidy: 2)
Coordinates:
    variant_position              (variants) int32 412MB dask.array<chunksize=(524288,), meta=np.ndarray>
    variant_contig                (variants) uint8 103MB dask.array<chunksize=(524288,), meta=np.ndarray>
    sample_id                     (samples) <U36 94kB dask.array<chunksize=(36,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy
Data variables:
    variant_allele                (variants, alleles) |S1 412MB dask.array<chunksize=(524288, 1), meta=np.ndarray>
    variant_filter_pass_funestus  (variants) bool 103MB dask.array<chunksize=(300000,), meta=np.ndarray>
    call_genotype                 (variants, samples, ploidy) int8 135GB dask.array<chunksize=(300000, 36, 2), meta=np.ndarray>
    call_GQ                       (variants, samples) int8 67GB dask.array<chunksize=(300000, 36), meta=np.ndarray>
    call_MQ                       (variants, samples) float32 270GB dask.array<chunksize=(300000, 36), meta=np.ndarray>
    call_AD                       (variants, samples, alleles) int16 540GB dask.array<chunksize=(300000, 36, 4), meta=np.ndarray>
    call_genotype_mask            (variants, samples, ploidy) bool 135GB dask.array<chunksize=(300000, 36, 2), meta=np.ndarray>
Attributes:
    contigs:  ('2RL', '3RL', 'X')

The arrays within this dataset are backed by [Dask arrays](https://docs.dask.org/en/latest/array.html), and can be accessed as shown below.

### SNP sites and alleles

We have called SNP genotypes in all samples at all positions in the genome where the reference allele is not "N". Data on this set of genomic positions and alleles for a given chromosome (e.g., 2RL) can be accessed as [Dask arrays](https://docs.dask.org/en/latest/array.html) as follows.

In [7]:
pos = ds_snps["variant_position"].data
pos

dask.array<array, shape=(102882611,), dtype=int32, chunksize=(524288,), chunktype=numpy.ndarray>

In [8]:
alleles = ds_snps["variant_allele"].data
alleles

dask.array<concatenate, shape=(102882611, 4), dtype=|S1, chunksize=(524288, 3), chunktype=numpy.ndarray>

Data can be loaded into memory as a [NumPy array](https://numpy.org/doc/stable/user/absolute_beginners.html) as shown in the following examples.

In [9]:
# read first 10 SNP positions into a numpy array
p = pos[:10].compute()
p

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int32)

In [10]:
# read first 10 SNP alleles into a numpy array
a = alleles[:10].compute()
a

array([[b'T', b'A', b'C', b'G'],
       [b'G', b'A', b'C', b'T'],
       [b'G', b'A', b'C', b'T'],
       [b'C', b'A', b'T', b'G'],
       [b'G', b'A', b'C', b'T'],
       [b'T', b'A', b'C', b'G'],
       [b'C', b'A', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'C', b'A', b'T', b'G'],
       [b'T', b'A', b'C', b'G']], dtype='|S1')

Here the first column contains the reference alleles, and the remaining columns contain the alternate alleles.

Note that a byte string data type is used here for efficiency. E.g., the Python code `b'T'` represents a byte string containing the letter "T", which here stands for the nucleotide thymine.

Note that we have chosen to genotype all samples at all sites in the genome, assuming all possible SNP alleles. Not all of these alternate alleles will actually have been observed in the `Af1` samples. To determine which sites and alleles are segregating, an allele count can be performed over the samples you are interested in. See the example below. 

### Site filters

SNP calling is not always reliable, and we have created some site filters to allow excluding low quality SNPs. 

Each set of site filters provides a "filter_pass" Boolean mask for each chromosome arm, where True indicates that the site passed the filter and is accessible to high quality SNP calling.

The site filters data can be accessed as dask arrays as shown in the examples below. 

In [ ]:
# access gamb_colu_arab site filters for chromosome 2RL as a dask array
filter_pass = ds_snps['filter_pass'].data
filter_pass

dask.array<array, shape=(102882611,), dtype=bool, chunksize=(300000,), chunktype=numpy.ndarray>

In [12]:
# read filter values for first 10 SNPs (True means the site passes filters)
f = filter_pass[:10].compute()
f

array([False, False, False, False, False, False, False, False, False,
       False])

### SNP genotypes

SNP genotypes for individual samples are available. Genotypes are stored as a three-dimensional array, where the first dimension corresponds to genomic positions, the second dimension is samples, and the third dimension is ploidy (2). Values coded as integers, where -1 represents a missing value, 0 represents the reference allele, and 1, 2, and 3 represent alternate alleles.

SNP genotypes can be accessed as dask arrays as shown below.

In [13]:
gt = ds_snps["call_genotype"].data
gt

dask.array<concatenate, shape=(102882611, 656, 2), dtype=int8, chunksize=(300000, 50, 2), chunktype=numpy.ndarray>

Note that the columns of this array (second dimension) match the rows in the sample metadata, if the same sample sets were loaded. I.e.:

In [ ]:
df_samples = adir1.sample_metadata(sample_sets="1.0")
gt = ds_snps["call_genotype"].data
len(df_samples) == gt.shape[1]

True

You can use this correspondance to apply further subsetting operations to the genotypes by querying the sample metadata. E.g.:

In [15]:
loc_funestus = df_samples.eval("taxon == 'funestus'").values
print(f"found {np.count_nonzero(loc_funestus)} funestus samples")

found 656 funestus samples


In [16]:
ds_snps_funestus = ds_snps.isel(samples=loc_funestus)
ds_snps_funestus

<xarray.Dataset> Size: 1TB
Dimensions:                       (variants: 102882611, alleles: 4,
                                   samples: 656, ploidy: 2)
Coordinates:
    variant_position              (variants) int32 412MB dask.array<chunksize=(524288,), meta=np.ndarray>
    variant_contig                (variants) uint8 103MB dask.array<chunksize=(524288,), meta=np.ndarray>
    sample_id                     (samples) <U36 94kB dask.array<chunksize=(36,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy
Data variables:
    variant_allele                (variants, alleles) |S1 412MB dask.array<chunksize=(524288, 1), meta=np.ndarray>
    variant_filter_pass_funestus  (variants) bool 103MB dask.array<chunksize=(300000,), meta=np.ndarray>
    call_genotype                 (variants, samples, ploidy) int8 135GB dask.array<chunksize=(300000, 36, 2), meta=np.ndarray>
    call_GQ                       (variants, samples) int8 67GB dask.array<chunksize=(300000, 36), meta=np.ndarray>
    call_MQ                       (variants, samples) float32 270GB dask.array<chunksize=(300000, 36), meta=np.ndarray>
    call_AD                       (variants, samples, alleles) int16 540GB dask.array<chunksize=(300000, 36, 4), meta=np.ndarray>
    call_genotype_mask            (variants, samples, ploidy) bool 135GB dask.array<chunksize=(300000, 36, 2), meta=np.ndarray>
Attributes:
    contigs:  ('2RL', '3RL', 'X')

Data can be read into memory as numpy arrays, e.g., read genotypes for the first 5 SNPs and the first 3 samples:

In [17]:
g = gt[:5, :3, :].compute()
g

array([[[-1, -1],
        [-1, -1],
        [-1, -1]],

       [[-1, -1],
        [-1, -1],
        [-1, -1]],

       [[-1, -1],
        [-1, -1],
        [-1, -1]],

       [[-1, -1],
        [-1, -1],
        [-1, -1]],

       [[-1, -1],
        [-1, -1],
        [-1, -1]]], dtype=int8)

If you want to work with the genotype calls, you may find it convenient to use [scikit-allel](http://scikit-allel.readthedocs.org/).
E.g., the code below sets up a genotype array.

In [19]:
# use the scikit-allel wrapper class for genotype calls
gt = allel.GenotypeDaskArray(ds_snps["call_genotype"].data)
gt

<GenotypeDaskArray shape=(102882611, 656, 2) dtype=int8>

## Example computation

Here's an example computation to count the number of segregating SNPs on the longest contig (KB672490) that also pass site filters. This may take a minute or two, because it is scanning genotype calls at millions of SNPs in hundreds of samples.

In [ ]:
# choose contig (longest contig)
region = "KB672490"
# choose site filter mask

# choose sample sets
sample_sets = ["1278-VO-TH-KOBYLINSKI-VMF00153"]

# access SNP calls
ds_snps = adir1.snp_calls(region=region, sample_sets=sample_sets)

# locate pass sites
loc_pass = ds_snps[f"filter_pass"].values

# perform an allele count over genotypes
gt = allel.GenotypeDaskArray(ds_snps["call_genotype"].data)
with ProgressBar():
    ac = gt.count_alleles(max_allele=3).compute()

# locate segregating sites
loc_seg = ac.is_segregating()

# count segregating and pass sites
n_pass_seg = np.count_nonzero(loc_pass & loc_seg)
n_pass_seg

[########################################] | 100% Completed | 30.1s


4779799

## Running larger computations

Please note that free cloud computing services such as Google Colab and MyBinder provide only limited computing resources. Thus although these services are able to efficiently read `Af1` data stored on Google Cloud, you may find that you run out of memory, or computations take a long time running on a single core. If you would like any suggestions regarding how to set up more powerful compute resources in the cloud, please feel free to get in touch via the [malariagen/vector-data GitHub discussion board](https://github.com/malariagen/vector-data/discussions).

## Feedback and suggestions

If there are particular analyses you would like to run, or if you have other suggestions for useful documentation we could add to this site, we would love to know, please get in touch via the [malariagen/vector-data GitHub discussion board](https://github.com/malariagen/vector-data/discussions).